# Задание на дифференцированный зачет

1. выбрать предметную область (это может быть один или несколько сайтов, посвященных одной тематике на выбор студента), нужно согласовать, создав отдельную followup дискуссию в своей теме на piazza
2. собрать по выбранной предметной области коллекцию документов (не менее 1000 документов),
3. реализовать в этой коллекции поиск, поисковые запросы должны быть устойчивы к опечаткам, а поисковая выдача должна ранжироваться,
4. для каждого документа из коллекции рекомендовать ранжированный перечень статей из Википедии.


# Описание предметной области

Компьтерные игры - данные с kaggle подборок, собранных по торговой площадке Steam.
Данные: id в steam, название, описание, разработчик (компания), публикатор (компания), жанр, теги, категории, кол-во загрузок и т.д.

# Предобработка

In [ ]:
import pandas as pd
import numpy as np

Загружаем сформированный датасет по ссылке из гугл диска и формируем dataframe

In [ ]:
!gdown 1Xbd5FwuQjCLInZxgBFXMtdHPfh3nLUum

Downloading...
From: https://drive.google.com/uc?id=1Xbd5FwuQjCLInZxgBFXMtdHPfh3nLUum
To: /content/steam_games.csv
100% 40.2M/40.2M [00:00<00:00, 85.7MB/s]


In [ ]:
df = pd.read_csv('steam_games.csv', sep=';')

<ipython-input-3-ce256c07b0d3>:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('steam_games.csv', sep=';')


Именуем колоки под удобную обработку

In [ ]:
df.columns = ['id', 'name', 'descr', 'developer', 'publisher', 'genre', 'tags', 'type', 'categories', 'downloads', 'positive_reviews',
       'negative_reviews', 'price', 'start_price', 'discount', 'ccu',
       'languages', 'platforms', 'release', 'age', 'website',
       'image']

В датафрейме представлено число позитивных и негативных оценок. Рассчитаем рейтинг игры ках отношение позитивных оценок к сумме оценок умноженное на шкалу рейтинга (мы будет оценивать по 5 бальной шкале). Удалим столбцы с числом оценок.

In [ ]:
scale = 5.0
df['rating'] = df['positive_reviews'] * scale / (df['positive_reviews'] + df['negative_reviews'])

In [ ]:
df = df.drop(columns=['positive_reviews', 'negative_reviews'])

Возраст представлен как int, в дальнейшем на потребуется строка. Загрузки представлены в формате ([от]..[до]). Пример: 500,000..1,000,000. Преобразуем в число и для аппроксимации занчения возьмем максимальное число.

In [ ]:
df['age'] = df['age'].apply(str)
start_list = df['downloads'].values.tolist()

In [ ]:
units = [unit.split(' .. ') for unit in start_list]
df['downloads'] = df['downloads'].str.split(' .. ').apply(lambda x: max([int(i.replace(',', '')) for i in x]))

Крооме того в датафрейме присутствуют записи с типом "Программное обеспечение". Нас же интересуют только игры.

In [ ]:
df = df[~(df['type'] != 'game')].reset_index(drop=True)

Отобрасываем ненужные колонки(тип, начальная цена, скидка, максимальный онлайн, сайт, изображение)

In [ ]:
df = df.drop(columns=['type', 'start_price', 'discount', 'ccu', 'website', 'image'])

Для дальнейшего ранжирования и поиска возраст не удобен в виде числа (тем более есть записи по типу MA 15+, 7+ и т.п.). Поэтому, используя категориальный признак, преобразуем все к единому формату.

In [ ]:
import re

def age_to_category(age):
    regex = r'(\d+|\d+\+|MA \d+\+)'
    match = re.search(regex, age)
    if match:
        age = match.group()
        if age == '0':
            return 'Infants'
        elif age.isdigit():
            age = int(age)
            if age < 6:
                return 'Toddlers'
            elif age < 13:
                return 'Children'
            elif age < 18:
                return 'Teens'
            elif age < 30:
                return 'Young Adults'
            elif age < 50:
                return 'Adults'
            else:
                return 'Seniors'
        elif age == 'MA 15+' or age.endswith('+'):
            return 'Parental Guidance'
    return None

df['age'] = [age_to_category(i) for i in df['age']]

Полученный датафрейм, состоит из 56690 игр.

In [ ]:
df

,id,name,descr,developer,publisher,genre,tags,categories,downloads,price,languages,platforms,release,age,rating
0,10,Counter-Strike,Play the world's number 1 online action game. ...,Valve,Valve,Action,"Action: 5426, FPS: 4831, Multiplayer: 3392, Sh...","Multi-player, Valve Anti-Cheat enabled, Online...",20000000,999,"English, French, German, Italian, Spanish - Sp...","windows, mac, linux",2000/11/1,Infants,4.874064
1,1000000,ASCENXION,ASCENXION is a 2D shoot 'em up game where you ...,IndigoBlue Game Studio,PsychoFlux Entertainment,"Action, Adventure, Indie","""Shoot Em Up"": 186, Metroidvania: 181, Bullet ...","Single-player, Partial Controller Support, Ste...",20000,999,"English, Korean, Simplified Chinese",windows,2021/05/14,Infants,4.218750
2,1000010,Crown Trick,"Enter a labyrinth that moves as you move, wher...",NEXT Studios,"Team17, NEXT Studios","Adventure, Indie, RPG, Strategy","Rogue-like: 268, Turn-Based Combat: 254, RPG: ...","Single-player, Partial Controller Support, Ste...",500000,599,"Simplified Chinese, English, Japanese, Traditi...",windows,2020/10/16,Infants,4.309534
3,1000030,"Cook, Serve, Delicious! 3?!","Cook, serve and manage your food truck as you ...",Vertigo Gaming Inc.,Vertigo Gaming Inc.,"Action, Indie, Simulation, Strategy","Typing: 221, Management: 213, Casual: 209, Dif...","Multi-player, Single-player, Co-op, Steam Achi...",200000,1999,English,"windows, mac",2020/10/14,Infants,4.659763
4,1000040,细胞战争,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,DoubleC Games,DoubleC Games,"Action, Casual, Indie, Simulation","Action: 22, Casual: 22, Indie: 21, Simulation: 20",Single-player,20000,199,Simplified Chinese,windows,2019/03/30,Infants,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55685,999880,NASA's Exoplanet Excursions,Take VR journeys through the amazing TRAPPIST-...,"Caltech/IPAC, NASA's Jet Propulsion Laboratory","Caltech/IPAC, NASA's Jet Propulsion Laboratory",Education,"Education: 22, VR: 6, Space: 5",NaN,20000,0,English,windows,2019/04/18,Infants,4.000000
55686,999890,Bruken,"Explore a large spherical world, enjoying all ...",FlairBot Games,FlairBot Games,"Adventure, Casual, Indie","Casual: 21, Indie: 21, Adventure: 20",Single-player,20000,299,English,windows,2019/04/11,Infants,5.000000
55687,999900,Studio by RADiCAL,"RADiCAL Studio is a real-time, unlimited motio...",RADiCAL,"RADiCAL Solutions, LLC",Animation & Modeling,"Animation & Modeling: 25, 3D: 13",NaN,50000,0,English,windows,2019/01/4,Infants,1.935484
55688,999930,Fantasy Sino-Japanese War 幻想甲午,You unfortunately passed through the Sino-Japa...,张八万工作室,张八万工作室,"Indie, RPG, Strategy, Early Access","Early Access: 21, Indie: 20, RPG: 20, Strategy...","Single-player, Partial Controller Support, Ste...",20000,499,"English, Simplified Chinese, Traditional Chinese",windows,2019/01/11,Infants,2.500000


# Обработка

In [ ]:
!pip install langdetect tqdm pandarallel pymorphy2 pymorphy2-dicts-uk

In [ ]:
data = df.copy()

Обработаем датафрейм на наличие иероглифических языков, удалим записи, где они встречаются в описании. Для всех остальных записей определим язык.

In [ ]:
import unicodedata
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

def detect_lang(text):
    from langdetect.lang_detect_exception import LangDetectException
    from langdetect import detect
    if pd.isna(text):
        return None
    try:
        if len(text) > 2 and any(c.isalpha() for c in text):
            lang = detect(text)
        else:
            lang = None
    except LangDetectException:
        lang = None
    return lang

def contains_hieroglyphs(text):
    if isinstance(text, str):
        for char in text:
            try:
                if 'CJK' in unicodedata.name(char):
                    return True
            except ValueError:
                pass
    return False

data['contains_hieroglyphs'] = data['descr'].parallel_apply(contains_hieroglyphs)
data = data.drop(data[data['contains_hieroglyphs'] == True].index)

data['lang'] = data['descr'].parallel_apply(detect_lang)


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
data.drop(['contains_hieroglyphs'], axis=1, inplace=True)
data = data.dropna(subset=['lang'])

In [ ]:
data_processing = data.copy()

Лемматизируем, токенизируем, удалим стоп слова. Результат сохраним в отдельную колонку

In [ ]:
import nltk.data
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2
import re
from tqdm import tqdm
import pandas as pd

morph = pymorphy2.MorphAnalyzer()
stops = [stopword for lang in stopwords.fileids() for stopword in stopwords.words(lang)]
tokenizer = RegexpTokenizer(r'\w+')


def process_text(text, tokenizer):
    words_list = [w for w in tokenizer.tokenize(text) if w.lower() not in stops and re.match(r'^[a-zA-Zа-яА-Я]+$', w) and not pd.isna(w)]
    return [morph.parse(w)[0].normal_form for w in words_list]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
not_show = ['id', 'lang']

processed_text_list = []
with tqdm(total=len(data_processing)) as pbar:
    for i, row in data_processing.iterrows():
        text = ''
        for c in data_processing.columns.values:
            if c not in not_show and not pd.isna(row[c]):
                text += f'{c}: {row[c]}. '
        processed_text = " ".join(process_text(text, tokenizer))
        processed_text_list.append(processed_text)
        pbar.update(1)
        pbar.set_description(f"Processed ")

data_processing['processed_text'] = processed_text_list

Processed : 100%|██████████| 54908/54908 [21:52<00:00, 41.83it/s]


In [ ]:
data_processing['id'] = data_processing['id'].apply(int)

In [ ]:
df_new = data_processing[['id', 'name', 'processed_text']].reset_index(drop=True)

Выведем получившийся датасет

In [ ]:
df_new

,id,name,processed_text
0,10,Counter-Strike,counter strike descr play world number online ...
1,1000000,ASCENXION,ascenxion descr ascenxion shoot game explore f...
2,1000010,Crown Trick,crown trick descr enter labyrinth moves move m...
3,1000030,"Cook, Serve, Delicious! 3?!",cook serve delicious descr cook serve manage f...
4,1000080,Zengeon,zengeon descr zengeon anime infused action rpg...
...,...,...,...
54903,999880,NASA's Exoplanet Excursions,nasa exoplanet excursions descr vr journeys am...
54904,999890,Bruken,bruken descr explore large spherical world enj...
54905,999900,Studio by RADiCAL,studio radical descr radical studio real time ...
54906,999930,Fantasy Sino-Japanese War 幻想甲午,fantasy sino japanese descr unfortunately pass...


Добавим ссылку на игру в Steam

In [ ]:
data_processing['steam_link'] = ['http://store.steampowered.com/app/' + str(i) for i in data_processing['id']]

Сохраним полный датасет на диск

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_processing.to_csv('/content/drive/MyDrive/exam_analyse/games.csv', index=False)

# Поиск

Добавим усточивость к опечаткам с помощью spellservice от Yandex

In [ ]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

yandex_api_base = "https://speller.yandex.net/services/spellservice.json"
check_text_suffix = "/checkText?text={}"

def fix_typo_word(uncorrect_word):
    req = requests.get(yandex_api_base + check_text_suffix.format(uncorrect_word))
    if req.ok and req.json() and len(req.json()[0]['s']):
        return req.json()[0]['s'][0]
    return uncorrect_word

def fix_typo_sentence(uncorrect_sent):
    with ThreadPoolExecutor(max_workers=8) as executor:
        fixed_words = list(executor.map(fix_typo_word, uncorrect_sent.split()))
    return ' '.join(fixed_words)

In [ ]:
def tokenize(text):
  return " ".join(process_text(text, tokenizer))

Используем TF-IDF для поска похожих документов, ускоренный с помощью преобразования в разреженные матрицы.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
import scipy.sparse as sp
data_processing = pd.read_csv('/content/drive/MyDrive/exam_analyse/games.csv')
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3))
tfidf_vector = vectorizer.fit_transform(data_processing['processed_text'])
tfidf_matrix = sp.csr_matrix(tfidf_vector.T)

In [ ]:
def simularitiesDocs(text, n=3, sortby='rating', asc=False):
    text = fix_typo_sentence(text)
    print("С учетом исправленных опечаток - ", text)
    text = tokenize(text)
    print("Токенизация: ", text)

    v = vectorizer.transform([text])
    simularities = cosine_similarity(v, tfidf_matrix.T)
    cols_to_exclude = ['id', 'lang', 'processed_text']
    s = np.argsort(-simularities, axis=1)[:, :n]
    res = data_processing.iloc[s[0]].sort_values(sortby, ascending=asc)
    return res.loc[:, ~res.columns.isin(cols_to_exclude)].reset_index(drop=True)

In [ ]:
data_processing.columns

Index(['id', 'name', 'descr', 'developer', 'publisher', 'genre', 'tags',
       'categories', 'downloads', 'price', 'languages', 'platforms', 'release',
       'age', 'rating', 'lang', 'processed_text', 'steam_link'],
      dtype='object')

Осуществим примеры поиска

In [ ]:
print("Найти топ 10 игр по загрузкам, поддерживающих русский язык")
simularitiesDocs('languages: Russian', 10, 'downloads')

Найти топ 10 игр по загрузкам, поддерживающих русский язык
С учетом исправленных опечаток -  languages: Russian
Токенизация:  languages russian


,name,descr,developer,publisher,genre,tags,categories,downloads,price,languages,platforms,release,age,rating,steam_link
0,Cat or Bread?,A game about cat and bread.,Casioqu,Casioqu,"Casual, Indie","Casual: 226, Indie: 26, Psychological Horror: 14, Memes: 13, Cats: 11, Horror: 5","Single-player, Steam Trading Cards",1000000,99,Russian,windows,2017/05/8,Infants,4.521858,http://store.steampowered.com/app/574820
1,Пацанский цитатник / Russian Test,Russian Test is simple test about some specific russian culture in some places,Easy game,Game for people,"Action, Casual, Indie","Casual: 23, Indie: 23, Action: 21","Single-player, Steam Achievements",50000,99,"Russian, English",windows,2019/06/25,Infants,3.200000,http://store.steampowered.com/app/1088880
2,Cold Winter Morning,&quot;Is this justice?&quot;,Progress Game,Progress Programme,"Adventure, Indie, RPG","Adventure: 106, Indie: 106, RPG: 105, Violent: 21, Singleplayer: 10, Historical: 9",Single-player,50000,199,Russian,windows,2020/01/26,Infants,5.000000,http://store.steampowered.com/app/1217440
3,Nogibator: Way Of Legs,(Tower Defence + Runner) / 2,"TheDreik, Xitilon",Phoenix Reborn Games,"Casual, Indie","Casual: 32, Indie: 32","Single-player, Steam Achievements",20000,299,"Russian, English",windows,2018/02/14,Infants,2.916667,http://store.steampowered.com/app/754330
4,Curse of the dungeon,You have to explore amazing places to get to the secret secrets of your soul.,Axe Games,Enoops,"Action, Adventure, Casual, Indie, RPG","Action: 22, Adventure: 22, RPG: 22, Casual: 21, Indie: 21",Single-player,20000,49,"Russian, English, Simplified Chinese",windows,2019/10/31,Infants,0.500000,http://store.steampowered.com/app/1174810
5,Ninja jump,"Jump from roof to roof, look at the scale, the higher the scale at the moment you touch the screen, the higher and further the jump!",D1ONE,D1ONE,"Casual, Indie","Casual: 26, Indie: 24",Single-player,20000,399,"Russian, English",windows,2018/02/28,Infants,1.818182,http://store.steampowered.com/app/800530
6,Chaos and the White Robot,[no description],SDF games,SDF games,"Action, Casual, Indie","Action: 23, Casual: 21, Indie: 21, Hand-drawn: 11","Single-player, Partial Controller Support",20000,99,"Russian, English, French, Italian, German, Spanish - Spain, Simplified Chinese, Polish, Portuguese - Portugal, Turkish, Ukrainian, Japanese",windows,2017/09/15,Infants,3.653846,http://store.steampowered.com/app/698950
7,MOMO.EXE 2,You have to make a deal with Momo to do the work for the boss. What will Momo ask in return?,Dymchick1,Dymchick1,"Action, Adventure, Indie","Adventure: 42, Indie: 41, Action: 32, Horror: 16, Memes: 14, Funny: 13, Singleplayer: 13","Single-player, Steam Achievements, Steam Cloud",20000,299,"Russian, English, Simplified Chinese, Ukrainian, Spanish - Latin America, Polish","windows, mac, linux",2018/10/5,Infants,4.189189,http://store.steampowered.com/app/946780
8,ROAD HOMEWARD 3 underwater world,ROAD HOMEWARD 3: underwater world is an adventure in which you and your team need to find a way out of the cave in order to get home.,OFF1C1AL,OFF1C1AL,Adventure,Adventure: 31,"Single-player, Steam Achievements",20000,199,"Russian, English",windows,2019/07/29,Infants,1.071429,http://store.steampowered.com/app/1106030
9,Kakuro,Kakuro is an outstanding puzzle game for strengthening the brain,screwyman,screwyman,"Casual, Indie","Casual: 67, Indie: 61, Puzzle: 41, Strategy: 36, 2D: 34, Cute: 27, Space: 25, Logic: 24, Relaxing: 22, Singleplayer: 20","Single-player, Steam Achievements",20000,99,"Russian, English, French, German, Simplified Chinese, Japanese",windows,2019/07/29,Infants,1.666667,http://store.steampowered.com/app/1109740


In [ ]:
print("Найти топ 10 игр по рейтингу инди Жанра")
simularitiesDocs('genre: Indie', 10)

Найти топ 10 игр по рейтингу инди Жанра
С учетом исправленных опечаток -  genre: Indie
Токенизация:  genre indie


,name,descr,developer,publisher,genre,tags,categories,downloads,price,languages,platforms,release,age,rating,steam_link
0,JU,Ju is a simple puzzle game. Enjoy the game.,张公子,张公子,Indie,"Indie: 23, Puzzle: 5","Single-player, Steam Achievements, Steam Cloud",20000,99,"English, Simplified Chinese","windows, mac",2017/10/19,Infants,5.000000,http://store.steampowered.com/app/684430
1,Here Nya,The best game in the world!,ilxappp,VBS,Indie,Indie: 22,Single-player,20000,99,English,windows,2018/08/22,Infants,5.000000,http://store.steampowered.com/app/916150
2,M-Plan,"An exploration, but not amazing game.",QingHun,QingHun,Indie,Indie: 22,"Single-player, Steam Achievements",20000,0,"English, Simplified Chinese",windows,2018/09/21,Infants,5.000000,http://store.steampowered.com/app/935540
3,Clock Simulator,Click once a second. Every second. You are the clock.,Kool2Play,Kool2Play,Indie,"Indie: 51, Simulation: 18, Singleplayer: 13, Family Friendly: 10, Realistic: 10","Single-player, Steam Achievements, Steam Trading Cards",50000,0,"English, Polish, Russian, French","windows, mac, linux",2016/07/20,Infants,4.433498,http://store.steampowered.com/app/501120
4,Just One Color,A hard puzzle game.,Mask,Mask,Indie,Indie: 21,Single-player,20000,99,English,windows,2018/03/4,Infants,4.166667,http://store.steampowered.com/app/805840
5,Escape: VR,The main objective in this game is to find your way out.,JapanCo,Bitlock Studio,Indie,"Indie: 31, VR: 7","Single-player, Steam Trading Cards",200000,199,English,windows,2017/04/12,Infants,4.084967,http://store.steampowered.com/app/607440
6,Tank Hero VR,Come and enjoy the Tank VR journey!!!,ED,ED,"Indie, RPG, Simulation, Strategy","Simulation: 23, Indie: 22, RPG: 22, Strategy: 22, VR: 11",Single-player,20000,299,English,windows,2016/08/25,Infants,2.750000,http://store.steampowered.com/app/502810
7,Skinny,Skinny is a weak horror game.,lol,lol,Indie,"Indie: 31, Horror: 6","Single-player, Steam Achievements",20000,299,English,windows,2019/07/5,Infants,2.734375,http://store.steampowered.com/app/977340
8,Player One,"3D Game, Thirdperson Platformer",Prime2Gold,Prime2Gold,"Indie, Early Access","Indie: 31, Early Access: 21",Single-player,20000,199,English,windows,2019/10/10,Infants,2.500000,http://store.steampowered.com/app/1160190
9,Near Midnight,Taste the fear like never before,Indie Dynamics,Indie Dynamics,Indie,"Indie: 21, Horror: 5",Single-player,20000,99,English,windows,2016/10/13,Infants,2.200000,http://store.steampowered.com/app/522590


In [ ]:
print("Найти игру Counter Strike и две похожих на нее")
simularitiesDocs('Counter Strike', 3)

Найти игру Counter Strike и две похожих на нее
С учетом исправленных опечаток -  Counter Strike
Токенизация:  counter strike


,name,descr,developer,publisher,genre,tags,categories,downloads,price,languages,platforms,release,age,rating,steam_link
0,Counter-Strike: Source,Counter-Strike: Source blends Counter-Strike's award-winning teamplay action with the advanced technology of Source™ technology.,Valve,Valve,Action,"Shooter: 5259, Action: 3147, FPS: 3120, Multiplayer: 2496, Team-Based: 1674, First-Person: 1470, Tactical: 1328, Competitive: 1320, PvP: 1011, e-sports: 937, Military: 749, War: 639, Strategy: 513, Moddable: 492, Singleplayer: 389, Difficult: 385, Atmospheric: 299, Survival: 280, Simulation: 242, Sandbox: 203","Multi-player, Valve Anti-Cheat enabled, Stats, Includes Source SDK, Steam Achievements, Steam Cloud, Cross-Platform Multiplayer",20000000,999,"English, French, German, Italian, Japanese, Korean, Spanish - Spain, Russian, Simplified Chinese, Traditional Chinese, Thai, Turkish","windows, mac, linux",2004/11/1,Infants,4.809399,http://store.steampowered.com/app/240
1,Counter-Strike: Condition Zero,"With its extensive Tour of Duty campaign, a near-limitless number of skirmish modes, updates and new content for Counter-Strike's award-winning multiplayer game play, plus over 12 bonus single player missions, Counter-Strike: Condition Zero is a tremendous offering of single and multiplayer content.",Valve,Valve,Action,"Action: 1365, FPS: 1019, Shooter: 758, Multiplayer: 614, First-Person: 466, Singleplayer: 401, Classic: 398, Tactical: 374, Team-Based: 317, Competitive: 305, Military: 178, Strategy: 177, Online Co-Op: 176, Adventure: 88, Survival: 70, Atmospheric: 62, Old School: 45, Open World: 43, Dark: 39, Simulation: 39","Multi-player, Single-player, Valve Anti-Cheat enabled",10000000,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean","windows, mac, linux",2004/03/1,Infants,4.560006,http://store.steampowered.com/app/80
2,Counter-Strike Nexon: Studio,"Counter-Strike Nexon: Studio is a Free to Play MMOFPS offering competitive PvP and PvE action including content from the original Counter-Strike and all new game modes, map creation function, weapons, and hordes of Zombies!","Valve Corporation, Nexon Korea Corporation",Nexon Korea Corporation,"Action, Casual, Free to Play, Massively Multiplayer","Free to Play: 2831, Zombies: 2164, Multiplayer: 1344, Sandbox: 1264, FPS: 1252, Shooter: 1220, Action: 1085, Gore: 803, First-Person: 799, Co-op: 723, Online Co-Op: 671, PvP: 665, Massively Multiplayer: 654, Singleplayer: 654, PvE: 652, Mod: 651, Funny: 642, Horror: 640, Walking Simulator: 640, Indie: 636","Multi-player, Single-player, Co-op, Captions available, MMO, Steam Achievements, Steam Trading Cards, In-App Purchases",10000000,0,"English, French, German, Russian, Turkish, Spanish - Spain",windows,2014/10/7,Infants,3.312674,http://store.steampowered.com/app/273110


In [ ]:
print("Найти 10 игр c жанром RPG или тегом Realistic, отсортированных по дате релиза")
simularitiesDocs('tags: Realistic, genre: RPG', 10, 'release')

Найти 10 игр c жанром RPG или тегом Realistic, отсортированных по дате релиза
С учетом исправленных опечаток -  tags: Realistic, genre: RPG
Токенизация:  tags realistic genre rpg


,name,descr,developer,publisher,genre,tags,categories,downloads,price,languages,platforms,release,age,rating,steam_link
0,MythBusters: The Game - Crazy Experiments Simulator,Be a MythBuster in front and behind the camera! The game is a first-person sim that adapts the famous Discovery show. Run crazy experiments and manage the production of episodes.,Byte Barrel,Movie Games S.A.,"Action, Adventure, Simulation","Realistic: 278, Simulation: 273, Puzzle: 271, Sandbox: 264, Immersive Sim: 263, 3D: 258, First-Person: 255, Logic: 253, Management: 249, Science: 248, Building: 245, Physics: 242, Crafting: 237, Inventory Management: 229, Singleplayer: 219, Funny: 204, Resource Management: 184, Destruction: 151, Action: 145, Adventure: 140","Single-player, Steam Achievements",20000,1299,"English, French, Italian, German, Spanish - Spain, Japanese, Korean, Polish, Portuguese - Brazil, Simplified Chinese",windows,2022/09/1,Infants,2.321429,http://store.steampowered.com/app/811550
1,BRINK Traveler,BRINK Traveler is a virtual travel experience that takes you to amazing natural locations in full 3D to feel like you're really there. Step into a postcard and visit some of the most breathtaking places on Earth today!,BRINK XR Inc.,BRINK XR Inc.,Adventure,"Realistic: 230, VR: 226, Experience: 221, Nature: 219, Exploration: 199, 6DOF: 192, Education: 190, Relaxing: 188, Atmospheric: 181, Casual: 177, 3D: 176, Adventure: 171, Story Rich: 137, Singleplayer: 136, America: 135, Historical: 129, Narration: 125, Simulation: 85, Software: 83, Walking Simulator: 82, VR Only: 1",Single-player,20000,999,"English, French, Italian, German, Spanish - Spain",windows,2021/09/9,Infants,4.494382,http://store.steampowered.com/app/1462520
2,G-MODEアーカイブス28 セパスチャンネル,If we lose one thing at a time. The world will fall apart.,G-MODE,G-MODE,RPG,"RPG: 63, JRPG: 38, Pixel Graphics: 34, Retro: 31, Story Rich: 28, Singleplayer: 22","Single-player, Steam Cloud",20000,499,"English, Not supported, Japanese",windows,2021/07/26,Infants,3.750000,http://store.steampowered.com/app/1671220
3,G-MODEアーカイブス01 フライハイトクラウディア,A JRPG whose story takes place in the sky!,G-MODE,G-MODE,RPG,"RPG: 63, JRPG: 38, Pixel Graphics: 34, Retro: 32, Story Rich: 28, Singleplayer: 22","Single-player, Steam Cloud",20000,499,"English, Not supported, Japanese",windows,2021/04/26,Infants,5.000000,http://store.steampowered.com/app/1580750
4,Starter Pack,It is an action open-world FPS RPG game made with only a starter pack in UE4.,Mika Pi,Mika Pi,RPG,"RPG: 72, Open World: 47, Sci-fi: 44, Atmospheric: 41, Minimalist: 38, Surreal: 32, Action RPG: 29, Arcade: 28, FPS: 27, 3D: 24, First-Person: 22, Realistic: 19, Futuristic: 18, Combat: 16, PvE: 14, Singleplayer: 12",Single-player,20000,299,English,windows,2021/02/8,Infants,5.000000,http://store.steampowered.com/app/1519570
5,鲸走：前篇 Whale Left Part 1,To become Alice's friend!,伍达忠,伍达忠,RPG,"RPG: 56, RPGMaker: 12, 2D: 11, Linear: 11, Pixel Graphics: 11",Single-player,20000,99,"English, Simplified Chinese",windows,2020/06/15,Infants,4.375000,http://store.steampowered.com/app/1308790
6,Koikoi,This boy is not alone. Because you are with him!,RadicalHyena,RadicalHyena,RPG,"RPG: 30, Indie: 5",Single-player,20000,1099,"English, Japanese","windows, mac",2018/07/18,Infants,4.166667,http://store.steampowered.com/app/763380
7,ルナティックドーン レジェンドパック,Choose your own adventure RPG. Second in the Artdink Legend Pack series. This game is only available in Japanese.,ARTDINK,Komodo,RPG,RPG: 38,Single-player,20000,3499,Japanese,windows,2015/01/8,Infants,4.473684,http://store.steampowered.com/app/338070
8,Rising Storm Game of the Year Edition,Also contains all multi-player content from Red Orchestra 2: Heroes of Stalingrad,"Tripwire Interactive, Anti Matter Games",Tripwire Interactive,Action,"Realistic: 378, World War II: 271, FPS: 250, Action: 223, Multiplayer: 218, Tactical: 170, First-Person: 160, War: 127, Shooter: 126, Historical: 108, Military: 104, Difficult: 89, Simulation: 83, Team-

# Википедия

Скачиваем данные в dataframe

In [ ]:
import pandas as pd
data_wiki = pd.read_csv('/content/drive/MyDrive/exam_analyse/games.csv')

Удаляем служебные (заголовки наших колонок, которые мы добавили для лучшего поиска) слова из текста.

In [ ]:
def remove_substrings(text, substrings):
    words = text.split()
    words = [word for word in words if word not in substrings]
    return ' '.join(words)
substrings_to_remove = data_wiki.columns.tolist()

In [ ]:
data_wiki['processed_text'] = data_wiki['processed_text'].apply(remove_substrings, args=(substrings_to_remove,))

Для токенов возьмем только существительные, разделим токенизированный текст на отдельные слова и сохраним в numpy array для быстрой обработки с помощью функции apply

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
from tqdm import tqdm
import numpy as np
def tstring_to_tnarray(tokens_str):
  tokens = nltk.word_tokenize(tokens_str)
  pos_tags = nltk.pos_tag(tokens)
  return np.array(list(set([word for word, pos in pos_tags if pos.startswith('N')])))

tqdm.pandas(desc="Tokenizing texts")
data_wiki['processed_text_tokenized'] = data_wiki['processed_text'].progress_apply(tstring_to_tnarray)


Tokenizing texts: 100%|██████████| 54908/54908 [05:33<00:00, 164.73it/s]


Затем обратно преобразуем в List

In [ ]:
data_wiki['processed_text_tokenized'] = data_wiki['processed_text_tokenized'].apply(list)

Соберем воедино все токены

In [ ]:
def combine_lists(list_of_lists):
    result = []
    for lst in list_of_lists:
        result.extend(lst)
    return result

In [ ]:
all_words = combine_lists(data_wiki['processed_text_tokenized'].values.tolist())

In [ ]:
len(all_words)


1427839

Далее выберем 1000 наиболее часто встречающихся

In [ ]:
import collections
counter = collections.Counter(all_words)

In [ ]:
most_common_words = counter.most_common(1000)

In [ ]:
most_common_words_list = [word for word, count in most_common_words]

И пересечения со списком топа 1000 наиболее встречающихся будем считаь ключевыми словами для каждой игры

In [ ]:
data_wiki['keywords'] = [list(set(most_common_words_list).intersection(set(x))) for x in data_wiki['processed_text_tokenized']]

Выведем полученный результат

In [ ]:
data_wiki[['id', 'name', 'keywords', 'lang']]

,id,name,keywords,lang
0,10,Counter-Strike,"[action, windows, warfare, game, play, anti, v...",en
1,1000000,ASCENXION,"[adventure, action, windows, grow, side, cloud...",en
2,1000010,Crown Trick,"[adventure, action, protagonist, rpg, enemies,...",en
3,1000030,"Cook, Serve, Delicious! 3?!","[action, windows, cloud, hundreds, play, singl...",en
4,1000080,Zengeon,"[action, windows, adventure, protagonist, hund...",en
...,...,...,...,...
54903,999880,NASA's Exoplanet Excursions,"[education, windows, role, infants, system, je...",en
54904,999890,Bruken,"[gravity, forces, windows, adventure, world, i...",en
54905,999900,Studio by RADiCAL,"[windows, machine, infants, code, time, captur...",en
54906,999930,Fantasy Sino-Japanese War 幻想甲午,"[windows, fantasy, support, cloud, rpg, army, ...",en


Установим wikipedia и wikipedia-api. Первый будет выступать в качестве основного инструмента поиска, второй как default gateway.

In [ ]:
!pip install wikipedia wikipedia-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Будем пытаться найти страницу в википедии по ключевому слову, если не найдено то ищем в wikipedia-api, если там тоже не найдено добавляем суффикс _game для поиска только по игровой тематике. Если в ходе последнего шага не найдено - считаем что не нашлось статьи под ключевое слово.

Кроме того в фукнционале реализован поиск статей на языке, который определился автоматически в ходе Лемматизации, и дальнейшее ранжирование их по релевантности (релевантность считаем как сумму пересечений названия и описания страницы википедии с набором ключевых слов для поиска)

In [ ]:
import wikipedia
import wikipediaapi
import warnings
import re
warnings.filterwarnings("ignore", category=UserWarning, module='wikipedia')
def is_number_string(s):
    return s.isdigit()
def search_wiki(wiki, keyword, lang):
    try:
      page = wiki.page(keyword)
      wiki_page = {
          'keyword': keyword,
          'title': page.title,
          'description': page.summary,
          'link': page.url
      }
    except:
      try:
        another_page = wikipediaapi.Wikipedia(lang).page(keyword)
        wiki_page = {
            'keyword': keyword,
            'title': another_page.title,
            'description': another_page.summary,
            'link': another_page.fullurl
        }
      except:
        try:
          other_page = wiki.page(keyword + '_game')
          wiki_page = {
              'keyword': keyword + '_game',
              'title': other_page.title,
              'description': other_page.summary,
              'link': other_page.url
          }
        except:
          return None
    if is_number_string(wiki_page['title']):
      return None
    return wiki_page

def search_wiki_keywords(keywords, lang, name):
    wiki_pages = []
    wikipedia.set_lang(lang)
    keywords.add(name) #Добавляем название игры с перечень для поиска
    for keyword in tqdm(keywords):
        page = search_wiki(wikipedia, keyword, lang)
        if page is not None:
          title_matches = [word.lower() in page['title'].lower() for word in keywords]
          description_matches = [word.lower() in page['description'].lower() for word in keywords]
          total_matches = sum(title_matches) + sum(description_matches)
          wiki_pages.append({
              'keyword': keyword,
              'title': page['title'],
              'link': page['link'],
              'relevance': total_matches
          })
    return {d['keyword']: next((i for i in wiki_pages if i['title'] == d['title']), None) for d in wiki_pages}



Также можно ставить ограничение на топ n статей по релевантности. Для поиска напишем функцию, принимающую на вход id игры и n.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
def search_query(id, top_n):
  filtered = data_wiki[data_wiki['id'] == id].to_dict(orient='records')[0]
  filtered_keywords = set(filtered['keywords'])
  if(top_n > len(filtered_keywords)):
    raise ValueError("top_n не может быть больше количества ключевых слов")
  result = search_wiki_keywords(set(filtered['keywords']), filtered['lang'], filtered['name'])
  titles = set()
  links = set()
  res = []
  for k, v in result.items():
      if v['title'] not in titles and v['link'] not in links:
          titles.add(v['title'])
          links.add(v['link'])
          res.append([v['title'], v['link'], v['relevance']])
  return pd.DataFrame(res, columns=['article_title', 'article_link', 'article_relevance']) \
         .sort_values(by='article_relevance', ascending=False)[:top_n].reset_index(drop=True)

Получим рекомендацию по статьям из википедии для игры с id = 811550 (язык английский)

In [ ]:
search_query(811550, top_n = 10)

100%|██████████| 29/29 [00:30<00:00,  1.04s/it]


,article_title,article_link,article_relevance
0,Game,https://en.wikipedia.org/wiki/Game,8
1,achievements,https://en.wikipedia.org/wiki/Achievement,4
2,Resource,https://en.wikipedia.org/wiki/Resource,4
3,Puzzle,https://en.wikipedia.org/wiki/Puzzle,4
4,spain,https://en.wikipedia.org/wiki/Spain,4
5,Inventory,https://en.wikipedia.org/wiki/Inventory,4
6,Management,https://en.wikipedia.org/wiki/Management,4
7,science,https://en.wikipedia.org/wiki/Science,4
8,physics,https://en.wikipedia.org/wiki/Physics,3
9,Brazil,https://en.wikipedia.org/wiki/Brazil,3


Получим рекомендацию по статьям из википедии для игры с id = 1455490 (язык словацкий)

In [ ]:
search_query(1455490, top_n = 10)

100%|██████████| 21/21 [00:14<00:00,  1.47it/s]


,article_title,article_link,article_relevance
0,Steam (program),https://sk.wikipedia.org/wiki/Steam_(program),5
1,Second Toughest in the Infants,https://sk.wikipedia.org/wiki/Second_Toughest_in_the_Infants,4
2,Quartier du Combat,https://sk.wikipedia.org/wiki/Quartier_du_Combat,4
3,Adobe Flash Player,https://sk.wikipedia.org/wiki/Adobe_Flash_Player,4
4,First-person shooter,https://sk.wikipedia.org/wiki/First-person_shooter,4
5,Person of the Year,https://sk.wikipedia.org/wiki/Person_of_the_Year,3
6,Fantasy,https://sk.wikipedia.org/wiki/Fantasy,3
7,Exploration Flight Test-1,https://sk.wikipedia.org/wiki/Exploration_Flight_Test-1,2
8,Jódovaný povidón,https://sk.wikipedia.org/wiki/J%C3%B3dovan%C3%BD_povid%C3%B3n,2
9,Online,https://sk.wikipedia.org/wiki/Online,2


Получим рекомендацию по статьям из википедии для игры с id = 1003960 (язык французский)

In [ ]:
search_query(1003960, top_n = 8)

100%|██████████| 9/9 [00:08<00:00,  1.09it/s]


,article_title,article_link,article_relevance
0,adventure,https://fr.wikipedia.org/wiki/Adventure,2
1,Microsoft Windows,https://fr.wikipedia.org/wiki/Microsoft_Windows,2
2,support,https://fr.wikipedia.org/wiki/Support,2
3,player,https://fr.wikipedia.org/wiki/Player,2
4,indie,https://fr.wikipedia.org/wiki/Indie,2
5,De Profundis,https://fr.wikipedia.org/wiki/De_profundis,2
6,Platform controller hub,https://fr.wikipedia.org/wiki/Platform_controller_hub,2
7,Éditions Points,https://fr.wikipedia.org/wiki/%C3%89ditions_Points,2


# Вывод

Таким образом, был обработан датасет игр из Steam для поиска, а также для рекомендации статей из википедии, используя ключевые слова. На выходе получился удобный иснтрумент, использующий различные подходы для обработки текстовых данных